# Plotlyを使ってGanttを作ってみる。 
2023/7/29

事前準備ライブラリのインストール
参考になるURL

In [ ]:
## Package Install
## ターミナルで以下を実行する。
# pip install -U plotly
# pip install -U pydbc
# pip install -U matplotlib


## URL
# https://plotly.com/graphing-libraries/ # plotly library introduction

必要なライブラリを有効化する

In [2]:
import pandas as pd 
import pyodbc
import plotly.express as px
import plotly.io as pio
import matplotlib.pyplot as plt
import numpy as np

DBからDFにデータを展開する(sqlserver)

In [4]:
# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
server = 'yutakaPC\SQLEXPRESS' 
database = 'MYDB' 
username = 'DB_User' 
password = 'DB_User'  
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
# select 26 rows from SQL table to insert in dataframe.
query = "select * from [MYDB].[TRN_LOT_PROCESS] order by Resource,Start,Finish,Object;"
df = pd.read_sql(query, cnxn)
df['Key'] = df['Resource'] + '-' + df['Object']
df['Task'] = df['Object'] 
df.to_csv('../../00_data/schedule_sample.csv')
# print(df.sort_values(by=['Resource','Start'])['Key'])
# print(df.head(26))

C:\Users\nishi\AppData\Local\Temp\ipykernel_10124\3919769664.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)


Matpot使ってみる

ガントチャートを作りたい

In [6]:
# df = pd.DataFrame([
#     dict(Resource="A",Object="Job A", Status="End",Start='2023-07-29 10:00', Finish='2023-07-29 12:00'),
#     dict(Resource="A",Object="Job C", Status="End",Start='2023-07-30 13:05', Finish='2023-07-30 16:00'),
#     dict(Resource="A",Object="Job B", Status="End",Start='2023-07-29 11:15', Finish='2023-07-29 15:00'),
#     dict(Resource="A",Object="Job F", Status="Abend",Start='2023-07-29 11:15', Finish='2023-07-29 15:00'),
#     dict(Resource="A",Object="Job G", Status="Abend",Start='2023-07-29 11:15', Finish='2023-07-29 15:00'),
#     dict(Resource="A",Object="Job G", Status="Abend",Start='2023-07-29 14:15', Finish='2023-07-29 18:00'),
#     dict(Resource="A",Object="Job G", Status="Abend",Start='2023-07-29 19:15', Finish='2023-07-29 21:00'),
#     dict(Resource="A",Object="Job I", Status="Abend",Start='2023-07-29 21:15', Finish='2023-07-29 22:00'),
#     dict(Resource="A",Object="Job I", Status="Abend",Start='2023-07-29 14:15', Finish='2023-07-29 18:00'),
#     dict(Resource="A",Object="Job H", Status="Abend",Start='2023-07-29 11:15', Finish='2023-07-29 15:00'),
#     dict(Resource="B",Object="Job I", Status="Abend",Start='2023-07-29 13:12', Finish='2023-07-29 16:00'),
#     dict(Resource="B",Object="Job D", Status="End",Start='2023-07-31 13:12', Finish='2023-07-31 16:00'),
#     dict(Resource="C",Object="Job E", Status="Abend",Start='2023-07-29 14:03', Finish='2023-07-29 15:00')
# ])

# print(df.head(26))

fig = px.timeline(
    df,                                                 # DF to use 
    title="1st made gantt chart",                       # tittle of this graph
    x_start='Start', x_end='Finish',                    # the column name of DF to use star time & end time column
    y='Key',
    # y='Resource',
    # y='Object',                                           # the column name of DF to use row name  
    text='Object',                                        # the column name of DF to use display
    # pattern_shape='Object',
    opacity= 1,
    color_discrete_map={'RUN': 'Blue','WAIT':'limegreen','DOWN':'red'},
    # facet_row="Resource",
    # width=1200, height=600,
    color='Status'                                      # the column name of DF to use color section
)

# 画面向けのConfig
config = dict(
    {'scrollZoom': True,
        'displaylogo': False,
        'modeBarButtonsToAdd': [
        # 'drawline',  # 直線
        'drawopenpath',  # 自由な線
        # 'drawclosedpath',  # 閉じたフリーフォーム
        # 'drawcircle',  # 円
        # 'drawrect',  # 四角
        'eraseshape',  # 図形の削除
        'toggleSpikelines',  # ホバーしたプロットに垂直・水平な線
        'hoverclosest',  # 直近のプロット1点をホバー
        'hovercompare',  # 直近のプロットと同じxのプロット全点をホバー
        ]
        ,'edits': {
        'axisTitleText': True,  # 軸ラベルの内容の変更を可能に
        }
    }
)

# text font
fig.update_traces(width=1,textfont_size=10, textangle=0, textposition='inside', cliponaxis=False)

# 全体のレイアウト修正
fig.update_layout(
    # showlegend=False,
    legend_x=0.9,
    clickmode='event+select',
    dragmode='pan',

    xaxis_title="Time", 
    yaxis_title="",

    font_size=20,font_family = 'Arial',                 # グラフ全体のフォントサイズ変更
    hoverlabel_bgcolor='#DAEEED',                       # Change the hover tooltip background color to a universal light blue color.
    hoverlabel_font_size=15,
    # textangle = -180,
    # newshape_line_color='red',                          # 
    bargap=0,bargroupgap=0,
    barmode="group"                                     # チャートを並べて表示してくれる。                    
)

# 横軸のレイアウト修正
fig.update_xaxes(
    # setting of tick 
    showticklabels = True,side ="bottom",               # Place the tick labels on the top of the chart
    tickfont_size=15, tickangle = 0,
    tickformat='%Y/%m/%d %H:%M',                        # 横軸の書式を変更                                                        
    # setting of line
    showline = True,linecolor = 'Black',
    zeroline = True,
    showgrid = True,
    type="date",
    # rangeslider=dict(                                   # グラフ下にレンジスライダーを追加
    #     visible=True,
    #     thickness=0.05
    # ),                     
    # rangeselector=dict(                                 # グラフ上にレンジセレクターを追加
    #     # xanchor='center',x=1,
    #     # yanchor='bottom',y=0,        
    #     buttons=list([
    #         dict(
    #             count=3,                                # 数値
    #             label='3h',                             # ボタンラベル
    #             step='hour',                            # 単位
    #             stepmode='backward'                     # 現時点より以前
    #             # stepmode='todate'                     # 一番最初時点から   
    #         ),
    #         dict(
    #             count=6,                                # 数値
    #             label='6h',                             # ボタンラベル
    #             step='hour',                            # 単位
    #             stepmode='backward'                     # 現時点より以前
    #         ),
    #         dict(
    #             count=12,                               # 数値
    #             label='12h',                            # ボタンラベル
    #             step='hour',                            # 単位
    #             stepmode='backward'                     # 現時点より以前
    #         ),
    #         dict(
    #             count=1,                                # 数値
    #             label='1day',                           # ボタンラベル
    #             step='day',                             # 単位
    #             stepmode='backward'                     # 現時点より以前                
    #         ),
    #         dict(step='all')
    #     ])
    # )
)

# 縦軸のレイアウト修正
fig.update_yaxes(
    autorange='reversed',
    categoryarray=df.sort_values(by=['Resource','Start'])['Key'],       # 並べたい順序を指定する。
    categoryorder='array',                                              # Ccategoryarrayで指定した順番でソートするときに利用する。
    # categoryorder='category ascending',
    # gridcolor = 'Black',
    showline = True,linecolor = 'Black',
    zeroline = True,
    showgrid = True
)                  # 縦軸の向きを逆にする

fig.show()

pio.write_html(fig,file="./Gantt.html",config=config)

簡単なガントチャートならこっちでいいかも・・・

In [130]:
import plotly.figure_factory as ff

fig = ff.create_gantt(df, index_col='Status',bar_width=0.2)
fig.show()
